<br>

# DAEE

In [ ]:
import os
import time
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [ ]:
def predict_encoding(file_path, n_lines=30):
    """
    Predict Encoding determines the encoding of a given file, so that when reading the file (via 'Pandas', for example)
    characters, accents and symbols are spelled correctly.
    :param file_path: Name and path to file
    :param n_lines: Number of lines to read to set encoding. Parameter Optional. Default: 30 lines.
    :return: Encoding.
    """
    import chardet

    # Open the file as binary data
    with open(file_path, 'rb') as f:
        # Join binary lines for specified number of lines
        raw_data = b''.join([f.readline() for line in range(n_lines)])

    return chardet.detect(raw_data)['encoding']

<br>

# Read Data

In [ ]:
my_file = os.path.join('data', 'brutos', 'daee_arq8.txt')

enc = predict_encoding(my_file, n_lines=300)
enc

In [ ]:
df = pd.read_csv(
    os.path.join('data', 'brutos', 'daee_arq8.txt'),
    sep=';',
    skiprows=1,
    #encoding=enc,
    dtype={
        # Identificação
        #'QTD': 'quantidade',
        #'COD BACIA': 'cod_bacia',
        'NOME RIO/AQUIFERO': 'object',
        'COD RIO/POCO(DAEE)': 'object',    
        'DIST_FOZ(Km)': 'object',

        # Administrativo
        #'USUARIO': 'usuario',    
        #'AUTOS(DAEE)': 'autos_daee',
        #'USO': 'uso',
        #'SEQ': 'seq',
        #'FINALID_USO': 'finalidade_uso',
        #'SITUAC_ADMIN(DAEE)': 'situacao_administrativa',

        # Vazão
        #'VAZAO(M3/H)': 'vazao_m3h',
        #'HORA/DIA': 'hora_dia',
        #'DIA/MES': 'dia_mes',
        #'MES/ANO': 'mes_ano',

        # Coordenada
        'UTM_NORTE(Km)': 'object',
        'UTM_LESTE(Km)': 'object',
        'UTM_MC': 'object',
    }
)

df.dtypes

<br>

## Rename Columns

In [ ]:
# Colunas Originiais
#display(list(df.columns))

# Renomear Colunas
dict_columns = {
    # Identificação
    'QTD': 'quantidade',
    'COD BACIA': 'cod_bacia',
    'NOME RIO/AQUIFERO': 'nome_rio_aquifero',
    'COD RIO/POCO(DAEE)': 'cod_rio_poco',    
    'DIST_FOZ(Km)': 'dist_foz_km',
    
    # Administrativo
    'USUARIO': 'usuario',
    'AUTOS(DAEE)': 'autos_daee',
    'USO': 'uso',
    'SEQ': 'seq',
    'FINALID_USO': 'finalidade_uso',
    'SITUAC_ADMIN(DAEE)': 'situacao_administrativa',
    
    # Vazão
    'VAZAO(M3/H)': 'vazao_m3h',
    'HORA/DIA': 'hora_dia',
    'DIA/MES': 'dia_mes',
    'MES/ANO': 'mes_ano',
    
    # Coordenada
    'UTM_NORTE(Km)': 'utm_norte_km',
    'UTM_LESTE(Km)': 'utm_leste_km',
    'UTM_MC': 'utm_mc',
}

# Rename Columns
df.rename(columns=dict_columns, inplace=True, errors='ignore')
df

<br>

## Fix Columns

In [ ]:
# Drop Columns
df.drop('quantidade', axis=1, inplace=True, errors='ignore')

# Distância da Foz
df['dist_foz_km'] = df['dist_foz_km'].str.replace(',', '').astype(float)
df['dist_foz_km'] = pd.to_numeric(df['dist_foz_km'])

# Vazão
df['vazao_m3h'] = df['vazao_m3h'].str.replace(',', '').astype(float)
df['vazao_m3h'] = pd.to_numeric(df['vazao_m3h'])

# UTM Norte
df['utm_norte_km'] = df['utm_norte_km'].str.replace(',', '').astype(float)
df['utm_norte_km'] = pd.to_numeric(df['utm_norte_km'])
df['utm_norte'] = df['utm_norte_km'] * 1000
df['utm_norte'] = df['utm_norte'].astype(int)

# UTM Leste
df['utm_leste_km'] = df['utm_leste_km'].str.replace(',', '').astype(float)
df['utm_leste_km'] = pd.to_numeric(df['utm_leste_km'])
df['utm_leste'] = df['utm_leste_km'] * 1000
df['utm_leste'] = df['utm_leste'].astype(int)

# UTM MC
df['utm_mc'] = df['utm_mc'].str.replace(',', '').astype(float)
df['utm_mc'] = pd.to_numeric(df['utm_mc'])
df['utm_mc'] = df['utm_mc'].astype(int)

# Results
df

In [ ]:
list_columns = [
    'cod_bacia',
    #'nome_rio_aquifero',
    #'cod_rio_poco',
    #'dist_foz_km',
    #'usuario',
    #'autos_daee',
    #'uso',
    #'seq',
    #'finalidade_uso',
    #'situacao_administrativa',
    #'vazao_m3h',
    #'hora_dia',
    #'dia_mes',
    #'mes_ano',
    #'utm_norte_km',
    #'utm_leste_km',
    #'utm_mc',
    #'utm_norte',
    #'utm_leste'
]

for col in list_columns:
    try:
        df[col] = df[col].str.strip()
    except:
        pass
    print('>>> {}'.format(col))
    #display(set(df[col]))

<br>

# Erros

In [ ]:
# Erro: Horas por Dia maior que 24
mask = df['hora_dia'] <= 24
mask =~ mask
df.loc[mask, 'erro_data'] = 'Erro na hora/dia/mês/ano'

# Erro: Dias por mês maior que 31
mask = df['dia_mes']  <= 31
mask =~ mask
df.loc[mask, 'erro_data'] = 'Erro na hora/dia/mês/ano'

# Erro: Mês por Ano 12
mask = df['mes_ano']  <= 12
mask =~ mask
df.loc[mask, 'erro_data'] = 'Erro na hora/dia/mês/ano'

# Erro: Coordenadas sem definição
mask = df['utm_leste_km'] == 0
df.loc[mask, 'erro_coordenada'] = 'Erro na coordenada'

# Erro: Coordenadas sem definição
mask = df['utm_norte_km'] == 0
df.loc[mask, 'erro_coordenada'] = 'Erro na coordenada'

# Erro: Coordenadas sem definição de Meridicano Central
mask = df['utm_mc'] == 0
df.loc[mask, 'erro_coordenada'] = 'Erro na coordenada'

# Results
df

<br>

# Export

In [ ]:
df.to_csv(
    os.path.join('data', 'tab_daee.csv'),
    index=False,
)

In [ ]:
# Exclui Problemas
df = df[df['erro_data'].isna()]
df = df[df['erro_coordenada'].isna()]

# Export Clean
df.to_csv(
    os.path.join('data', 'tab_daee_clean.csv'),
    index=False,
)

In [ ]:
# Seleciona Amostra Aleatória
df = df.sample(n=1000)

# Export Clean
df.to_csv(
    os.path.join('data', 'tab_daee_exemplo.csv'),
    index=False,
)